In [ ]:
import os
import re
import openai

In [ ]:
# Define Azure endpoints (or OpenAI endpoints if using OpenAI directly)
AZURE_ORGANIZATION = "Your Azure Organization"

# Change the base to your region
openai.api_type = "azure"
openai.api_base = f"https://{AZURE_ORGANIZATION}-uk-south.openai.azure.com/"
openai.api_version = "2023-07-01-preview"

In [ ]:
def set_openai_uk():
    # Switch current resource to UK
    openai.api_base = f"https://{AZURE_ORGANIZATION}-uk-south.openai.azure.com/"
    openai.api_key = os.getenv("OPENAI_KEY_UK")
    print("OPENAI Key set to UK")

    
def set_openai_fr():
    # Switch current resource to FR
    openai.api_base = f"https://{AZURE_ORGANIZATION}-france-central.openai.azure.com/"
    openai.api_key = os.getenv("OPENAI_KEY_FR")
    print("OPENAI Key set to FR")

    
def set_openai_sw():
    # Switch current resource to SW
    openai.api_base = f"https://{AZURE_ORGANIZATION}-sweden.openai.azure.com/"
    openai.api_key = os.getenv("OPENAI_KEY_SW")
    print("OPENAI Key set to SW")
    
def resource_switcher():
    """Generator that switches between the available Azure OpenAI resources
    """
    i = 0
    # Define possible resources
    # Include one function (as above) for each available resource
    resources = [set_openai_uk, set_openai_sw, set_openai_fr]

    while True:
        # Switch the resource by calling one of the functions defined
        yield resources[i]()
        # Move pointer to the next resource, or back to the first if the last is reached
        i += 1
        if i == 3:
            i = 0

In [ ]:
prompt = "Your Prompt Here"
load_bearer = resource_switcher()

prompt_done = False

conversation = [{"role": "user", "content": prompt}]
output = None

number_of_retries = 0
max_number_of_retries = 3

while not prompt_done:
    try:
        # Call API with prompts and settings
        response = openai.ChatCompletion.create(
        engine="gpt_4",
        messages = conversation)

        # Get response with highest confidence (first one)
        output = response['choices'][0]['message']['content']
        
        # Example condition for prompt success
        example_condition = len(output) > 100
        
        if number_of_retries < max_number_of_retries:
            assert example_condition
        
        # Tip: Ask output to be generated as python dictionary, extract only the dictionary from the response
        result = eval(re.findall(r'{[^{}]*}', output)[0])
        prompt_done = True

    # Prompt has succeeded, but output does not meet condition
    except AssertionError as e:
        conversation.append({'role': 'assistant', 'content': output})
        conversation.append({'role': 'user', 'content': 'Provide an answer with at least a hundred characters.'})
        number_of_retries += 1
        
    # If prompt fails, try again with different resource
    except Exception as e:      
        next(load_bearer)
        print(f"Prompt Failed: {e}, switching resource to {openai.api_base}")
        number_of_retries += 1
        prompt_done = number_of_retries == max_number_of_retries
        
print(result)